In [59]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import sklearn.linear_model as lm
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector
import seaborn as sns

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

# 'racedate','seq','place','distance','course',
# 'horseNoA','horseNoB','horseIdA','horseIdB','drawA','drawB','jockeyA','jockeyB',
# 'jockeyACnt','jockeyBCnt','jockeyRelRiskA2B',
# 'drawACnt','drawBCnt','drawRelRiskA2B',
# 'sameHorseACnt','sameHorseBCnt','sameHorseRelRiskA2B',
# 'diffHorseACnt','diffHorseBCnt','diffHorseRelRiskA2B',
# 'result'
features=[
    #'jockeyACnt','jockeyBCnt',
   'jockeyRelRiskA2B',
     #     'drawACnt','drawBCnt',
    'drawRelRiskA2B'
 # 'sameHorseACnt','sameHorseBCnt',
  #  'sameHorseRelRiskA2B',
#,'diffHorseACnt','diffHorseBCnt',
#'diffHorseRelRiskA2B'
]

target_fea='result'
target=[target_fea]
sql = 'select raceclass,racedate,seq,racemeeting,distance,course,horseNoA,horseNoB,drawA,drawB,jockeyA,jockeyB'
for aFeature in features:
    sql = sql + "," + aFeature 
#sql = sql+ ","+ target_fea
sql = sql+ ",(finishTimeA - finishTimeB) as result, 1 as constant from survival_analysis" 
#sql = sql+" where racemeeting ='ST' and raceclass=4 and distance=1650 and course ='ALL WEATHER TRACK'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features,*target]
features = ['constant',*features]
dataset0 = pd.read_sql(sql,mydb)
#print(dataset0)
#allDates =['20190217']
raceInfoAll=dataset0[['racemeeting','raceclass','distance','course']]
uniqueRaceInfo= raceInfoAll.drop_duplicates()
#print(uniqueRaceInfo)
tableName='survival_analysis'

from time import localtime, strftime
updatedate = strftime("%Y%m%d%H%M%S", localtime())

In [58]:
 #regression model of lbw or finishTime
for index, row in uniqueRaceInfo.iterrows():
    racemeeting = row['racemeeting']
    raceclass=row['raceclass']
    distance=row['distance']
    course=row['course']
    print("forecast:"+racemeeting + "::"+str(raceclass)+"::"+str(distance)+"::"+course)
    dataset1=dataset0[(dataset0['racemeeting']==racemeeting)&(dataset0['raceclass']==raceclass)
                      &(dataset0['distance']==distance)&(dataset0['course']==course)]
    dataset_train = dataset1[(dataset1['racedate'] <= '20190401')]
    dataset_test =  dataset1[(dataset1['racedate'] > '20190401')]

    #print(dataset_train.shape[0])
    #print(dataset_test.shape[0])
    if(dataset_train.shape[0] == 0 or dataset_test.shape[0] ==0): 
        continue
    X_train=dataset_train[features]
    y_train=dataset_train[target] #lbw

    X_test=dataset_test[features]
    y_test=dataset_test[target] #lbw


    reg = lm.LinearRegression() # -0.1
    #reg = lm.Ridge(alpha=.5) # -0.1
    reg.fit(X_train,y_train)
    y_test_predicted=reg.predict(X_test)

    print(reg.coef_)

    #plt.scatter( y_test, y_test_predicted, color='black')
    #plt.show()
    
    #print("Mean squared error: %.2f"
    #        % mean_squared_error(y_test, y_test_predicted))
    # Explained variance score: 1 is perfect prediction
    #print('r2_score score: %.2f' % r2_score(y_test, y_test_predicted))
    print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
    #print('explained_variance_score: %.2f' % explained_variance_score(y_test, y_test_predicted))
    
    df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['predicted_finishTime'])
    dataset_test_toshow=dataset_test[['racedate','seq','racemeeting','distance','course',
                                      'horseNoA','horseNoB',target_fea]]
    dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
    dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['racedate','seq','horseNoA','horseNoB','predicted_finishTime'])
    #print(dataset_test_withPredicted)

    predicted_place=1
    previous_raceDate=''
    previous_seq=''
    for index, row in dataset_test_withPredicted.iterrows():
        mycursor = mydb.cursor()
        raceDate_str = str(row['racedate'])
        raceSeqOfDate =str(row['seq'])
        horseNoA = str(row['horseNoA'])
        horseNoB = str(row['horseNoB'])
        predicted_finishTime =row['predicted_finishTime']

        if predicted_finishTime < 0 :
            predicted_result = 1
        else :
            predicted_result = 2
        #print(raceDate_str +' '+ raceSeqOfDate + ' '+ str(predicted_result) +' '+ str(predicted_finishTime))

        ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
        sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_result="+ str(predicted_result) 
               + ", predicted_finishTime = "+ str(predicted_finishTime)
               + " WHERE racedate = "+ raceDate_str + " and seq = "+raceSeqOfDate 
               + " and horseNoA ='"+horseNoA+ "' "
               + " and horseNoB ='"+horseNoB+ "' ")

        #print(sql)
        mycursor.execute(sql)
        mydb.commit()
        #print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )

       # print("total record:", count)


forecast:ST::4::1000::TURF - "B" COURSE
forecast:ST::5::1600::TURF - "B" COURSE
forecast:ST::1::1200::TURF - "B" COURSE
forecast:ST::4::1200::TURF - "B" COURSE
forecast:ST::5::1200::TURF - "B" COURSE
forecast:ST::4::1400::TURF - "B" COURSE
forecast:ST::3::1000::TURF - "B" COURSE
forecast:ST::2::1400::TURF - "B" COURSE
forecast:ST::3::1200::TURF - "B" COURSE
forecast:ST::3::1400::TURF - "B" COURSE
forecast:HV::5::1200::TURF - "A" COURSE
forecast:HV::4::1650::TURF - "A" COURSE
forecast:HV::5::1650::TURF - "A" COURSE
forecast:HV::4::1000::TURF - "A" COURSE
forecast:HV::4::1200::TURF - "A" COURSE
forecast:HV::3::1200::TURF - "A" COURSE
forecast:HV::3::1650::TURF - "A" COURSE
forecast:HV::2::1200::TURF - "A" COURSE
forecast:ST::4::1200::TURF - "C" COURSE
forecast:ST::5::1000::TURF - "C" COURSE
forecast:ST::4::1600::TURF - "C" COURSE
forecast:ST::5::1400::TURF - "C" COURSE
forecast:ST::4::1400::TURF - "C" COURSE
forecast:ST::4::1200::ALL WEATHER TRACK
[[ 0.         -0.13871947  0.0237726 ]]


forecast:HV::3::2200::TURF - "C" COURSE
forecast:ST::0::1000::TURF - "B" COURSE
forecast:ST::3::2400::TURF - "B" COURSE
forecast:ST::5::2200::TURF - "C" COURSE
forecast:ST::4::2200::TURF - "C+3" COURSE
forecast:HV::2::1800::TURF - "C" COURSE
forecast:ST::0::1200::TURF - "A" COURSE
forecast:HV::3::2200::TURF - "A" COURSE
forecast:ST::0::1000::TURF - "C" COURSE
forecast:ST::4::2000::TURF - "A" COURSE
forecast:ST::0::1200::TURF - "C" COURSE
forecast:ST::2::2000::TURF - "C" COURSE
forecast:ST::5::1800::TURF - "A" COURSE
forecast:ST::5::1200::TURF - "A" COURSE
forecast:ST::4::1800::TURF - "A+3" COURSE
forecast:ST::5::1800::TURF - "B+2" COURSE
forecast:ST::3::1000::TURF - "B+2" COURSE
forecast:ST::5::1600::TURF - "C+3" COURSE
forecast:ST::5::2200::TURF - "C+3" COURSE
forecast:ST::5::1000::TURF - "A+3" COURSE
forecast:ST::3::2000::TURF - "B+2" COURSE
forecast:ST::2::1800::TURF - "A+3" COURSE
forecast:ST::1::1200::TURF - "A+3" COURSE
forecast:ST::1::1600::TURF - "A+3" COURSE
forecast:ST::2::14

forecast:ST::1::2000::TURF - "A" Course
forecast:HV::5::1200::TURF - "C+3" Course
